In [1]:
import json
import os

import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn_image as isns

In [2]:
from pynpoint import Pypeline
from pynpoint.processing.stacksubset import StackAndSubsetModule
from pynpoint.processing.resizing import RemoveLinesModule
from pynpoint.processing.psfsubtraction import PcaPsfSubtractionModule
from pynpoint.readwrite.hdf5writing import Hdf5WritingModule
from pynpoint.readwrite.fitswriting import FitsWritingModule

## Create the Pypeline

In [3]:
root_dir = Path("/home/ipa/quanz/user_accounts/mbonse/2023_S4/30_data/")

In [4]:
subfolder = [i  for i in root_dir.iterdir()]
subfolder

[PosixPath('/home/ipa/quanz/user_accounts/mbonse/2023_S4/30_data/HD22049_244_199_C-0065_B_'),
 PosixPath('/home/ipa/quanz/user_accounts/mbonse/2023_S4/30_data/HD22049_303_199_C-0065_C_'),
 PosixPath('/home/ipa/quanz/user_accounts/mbonse/2023_S4/30_data/HD22049_304_199_C-0065_C_'),
 PosixPath('/home/ipa/quanz/user_accounts/mbonse/2023_S4/30_data/HD22049_310_0100_C-0656_A_'),
 PosixPath('/home/ipa/quanz/user_accounts/mbonse/2023_S4/30_data/HD22049_351_096_C-0679_A_')]

In [5]:
tmp_database = subfolder[0]
(tmp_database / "PynPoint_database.hdf5").is_file()

True

In [6]:
pipeline = Pypeline(
    working_place_in=str(tmp_database),
    input_place_in=str(tmp_database),
    output_place_in=str(tmp_database))

PynPoint v0.10.1

A new version (0.10.0) is available!

Want to stay informed about updates, bug fixes, and new features?
Please consider using the 'Watch' button on the Github page:
https://github.com/PynPoint/PynPoint

Working place: /home/ipa/quanz/user_accounts/mbonse/2023_S4/30_data/HD22049_244_199_C-0065_B_
Input place: /home/ipa/quanz/user_accounts/mbonse/2023_S4/30_data/HD22049_244_199_C-0065_B_
Output place: /home/ipa/quanz/user_accounts/mbonse/2023_S4/30_data/HD22049_244_199_C-0065_B_

Database: /home/ipa/quanz/user_accounts/mbonse/2023_S4/30_data/HD22049_244_199_C-0065_B_/PynPoint_database.hdf5
Configuration: /home/ipa/quanz/user_accounts/mbonse/2023_S4/30_data/HD22049_244_199_C-0065_B_/PynPoint_config.ini

Number of CPUs: 20
Number of threads: not set


## Reduce the resolution

In [7]:
pipeline.get_tags()

['object_low_res',
 'object_low_res_s2',
 'object_low_res_s5',
 'object_selected',
 'psf_selected',
 'residual_mean',
 'residual_mean_stacked_2',
 'residual_mean_stacked_5',
 'residual_median',
 'residual_median_stacked_2',
 'residual_median_stacked_5']

In [8]:
pipeline.get_shape('object_selected')

(60004, 259, 259)

In [9]:
remove_lines = RemoveLinesModule(
    name_in="Reduce_resolution",
    image_in_tag="object_selected",
    image_out_tag="object_low_res",
    lines=(59, 59, 59, 59),
)

In [10]:
pipeline.add_module(remove_lines)

In [ ]:
pipeline.run_module("Reduce_resolution")


-----------------
RemoveLinesModule
-----------------

Module name: Reduce_resolution
Input port: object_selected (60004, 259, 259)
Deleting data and attributes: object_low_res


## Stack the frames for PCA

In [ ]:
stacking_2 = StackAndSubsetModule(
    name_in="stacking_2",
    image_in_tag="object_low_res",
    image_out_tag="object_low_res_s2",
    stacking=2,
    combine="mean")

stacking_5 = StackAndSubsetModule(
    name_in="stacking_5",
    image_in_tag="object_low_res",
    image_out_tag="object_low_res_s5",
    stacking=5,
    combine="mean")

In [ ]:
pipeline.add_module(stacking_2)
pipeline.add_module(stacking_5)

In [ ]:
pipeline.run_module("stacking_2")
pipeline.run_module("stacking_5")

## Export as HDF5 for applefy

In [ ]:
write_hdf5 = Hdf5WritingModule(
    name_in="Write_Hdf5",
    file_name=tmp_database.name + ".hdf5",
    output_dir=str(tmp_database),
    tag_dictionary = {
        "object_low_res_s5": "object_stacked_05",
        "object_low_res_s2": "object_stacked_02",
        "object_low_res": "object",
        "psf_selected": "psf_template"},
    keep_attributes=True,
    overwrite=True)

In [ ]:
pipeline.add_module(write_hdf5)
pipeline.run_module("Write_Hdf5")